In [1]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, CustomLibrary
import matplotlib.pyplot as plt

from generate_data import generate_tracking_data, generate_discrete_tracking_data
from decimal import Decimal

In [2]:
T = 1e-1
t = np.linspace(0, 20, 201)
x0 = [-0.1, 0.2, -0.1]
x0_val = [0.1, 0.1, 0]

x, x_dot = generate_tracking_data(t=t, x0=x0)
x_val, _ = generate_tracking_data(t=t, x0=x0_val)

xk = generate_discrete_tracking_data(t=t, x0=x0, T=T)
xk_val = generate_discrete_tracking_data(t=t, x0=x0_val, T=T)

In [7]:
pr = int(3 - np.log10(T))
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=3), # 2, 1
    # feature_library=FourierLibrary(n_frequencies=1), # 2, 1
    optimizer=ps.STLSQ(threshold=0.0001), # 0.001, 0.01, 0.1
    feature_names=[f'x{i+1}' for i in range(len(x[0]))],
    discrete_time=True
    )
model.fit(x=xk)
model.print(precision=pr)

(x1)[k+1] = 0.9123 x1[k] + 0.1000 x3[k] + 0.0470 x1[k]^2 + -0.0088 x1[k] x3[k] + -0.0019 x2[k]^2 + 0.3846 x1[k]^3 + -0.1000 x1[k]^2 x3[k]
(x2)[k+1] = 1.0000 x2[k] + 0.1000 x3[k]
(x3)[k+1] = -0.4208 x1[k] + 0.9604 x3[k] + -0.0470 x1[k]^2 + -0.3564 x1[k]^3


In [9]:
for x_num in range(len(x0)):
    q = 'Biblioteka funkcji & Próg & $\Dot{x}'
    print(f'{q}_{x_num+1}$ & $E_{x_num+1}$ \\\\')
    for i, library in enumerate([FourierLibrary(n_frequencies=2), FourierLibrary(n_frequencies=1), PolynomialLibrary(degree=3), PolynomialLibrary(degree=2), PolynomialLibrary(degree=1)]):
        print(f'\\hline')
        for threshold in range(3):
            threshold = 10**(threshold-4) if i >= 2 else 100*10**(threshold-4)
            threshold = T * threshold
            name = ['Trygonometryczna (st. 2)', 'Trygonometryczna (st. 1)', 'Wielomiany (st. 3)', 'Wielomiany (st. 2)', 'Liniowa']
            model = ps.SINDy(
                feature_library=library,
                optimizer=ps.STLSQ(threshold=threshold),
                feature_names=[f'x{i+1}' for i in range(len(x0))],
                discrete_time=True
                )
            model.fit(x=xk)
            try:
                x_sim = model.simulate(x0=x0_val, t=201)
                mse = ((x_sim - xk_val)**2).mean(axis=0)
                E = '%.3E' % Decimal(str(mse[x_num]))
            except:
                E = '\infty'
            coeffs = ' + '.join(['%.3E' % Decimal(str(coeff))+' '+model.get_feature_names()[i] for i, coeff in enumerate(model.coefficients()[x_num]) if abs(model.coefficients()[x_num][i]) > threshold])
            if len(coeffs.split(' + ')) > 2:
                eq = (coeffs.split(' + ')[0] + ' + ' + coeffs.split(' + ')[1] + '\dots').replace(' 1 +', ' +')
            else:
                eq = coeffs
            if len(coeffs) == 0:
                eq = '0,000'
            eq = eq.replace(' 1 +', ' +').replace('.', ',').replace('+ -', '- ').replace('sin', '\sin').replace('cos', '\cos').replace('(1 x1)', '(x_1)').replace('(1 x2)', '(x_2)').replace('x2', 'x_2').replace('(1 x3)', '(x_3)').replace('x3', 'x_3').replace('x1', 'x_1').replace('(1 u)', '(u)')
            for pow in range(1, 10):
                eq = eq.replace('E+00', '')
                E = E.replace('E+00', '')
                eq = eq.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
                E = E.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
            print(f"{name[i]} & {(str(threshold).replace('.', ',') + ' &').replace(',0 &', ' &')} ${eq}$ & ${E.replace('.', ',')}$ \\\\")
    print('\n\n')

Biblioteka funkcji & Próg & $\Dot{x}_1$ & $E_1$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $1,813 \sin(x_1) - 6,217 \cos(x_1)\dots$ & $1,087\cdot 10^{-4}$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $1,220 \sin(x_1) + 7,758\cdot 10^{-1} \sin(x_2)\dots$ & $4,275\cdot 10^{-5}$ \\
Trygonometryczna (st. 2) & 0,1 & $9,591 \sin(x_1) - 4,347 \sin(2 x_1)$ & $4,353\cdot 10^{-4}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $9,150\cdot 10^{-1} \sin(x_1) + 1,290\cdot 10^{-2} \cos(x_1)\dots$ & $3,029\cdot 10^{-4}$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $9,147\cdot 10^{-1} \sin(x_1) + 9,999\cdot 10^{-2} \sin(1 x3)$ & $6,452\cdot 10^{-7}$ \\
Trygonometryczna (st. 1) & 0,1 & $9,220\cdot 10^{-1} \sin(x_1)$ & $4,484\cdot 10^{-4}$ \\
\hline
Wielomiany (st. 3) & 1e-05 & $9,123\cdot 10^{-1} x_1 + 1,000\cdot 10^{-1} x3\dots$ & $7,176E-26$ \\
Wielomiany (st. 3) & 0,0001 & $9,123\cdot 10^{-1} x_1 + 1,000\cdot 10^{-1} x3\dots$ & $7,176E-26$ \\
Wielomiany (st. 3) & 0,001 & $-1,494\cdot

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Wielomiany (st. 2) & 0,001 & $4,074\cdot 10^{-3} + 1,052 x_1\dots$ & $1,518\cdot 10^{-3}$ \\
\hline
Liniowa & 1e-05 & $6,678\cdot 10^{-4} + 9,140\cdot 10^{-1} x_1\dots$ & $8,723\cdot 10^{-7}$ \\
Liniowa & 0,0001 & $6,678\cdot 10^{-4} + 9,140\cdot 10^{-1} x_1\dots$ & $8,723\cdot 10^{-7}$ \\
Liniowa & 0,001 & $9,140\cdot 10^{-1} x_1 - 2,854\cdot 10^{-3} x_2\dots$ & $8,723\cdot 10^{-7}$ \\



Biblioteka funkcji & Próg & $\Dot{x}_2$ & $E_2$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $-2,544\cdot 10^{-3} \sin(x_1) + 1,563\cdot 10^{-1} \cos(x_1)\dots$ & $1,201\cdot 10^{-2}$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $6,921\cdot 10^{-2} \sin(x_1) + 4,461 \cos(x_1)\dots$ & $1,355\cdot 10^{-2}$ \\
Trygonometryczna (st. 2) & 0,1 & $5,450\cdot 10^{-1} \sin(x_2) + 2,450\cdot 10^{-1} \sin(2 x_2)$ & $7,021\cdot 10^{-2}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $-1,791\cdot 10^{-2} \cos(x_1) + 1,033 \sin(x_2)\dots$ & $2,471\cdot 10^{-1}$ \\
Trygonometryczna (st. 1) & 0,010000000000000

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Liniowa & 0,001 & $1,000 x_2 + 1,000\cdot 10^{-1} x3$ & $9,679\cdot 10^{-3}$ \\



Biblioteka funkcji & Próg & $\Dot{x}_3$ & $E_3$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $-1,275 \sin(x_1) - 2,557\cdot 10^{-2} \cos(x_1)\dots$ & $6,375\cdot 10^{-4}$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $-4,252\cdot 10^{-1} \sin(x_1) - 6,453\cdot 10^{-1} \sin(x_2)\dots$ & $3,273\cdot 10^{-4}$ \\
Trygonometryczna (st. 2) & 0,1 & $7,191 \sin(1 x3) - 3,141 \sin(2 x3)$ & $1,273\cdot 10^{-3}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $-4,234\cdot 10^{-1} \sin(x_1) + 5,272\cdot 10^{-2} \cos(x_1)\dots$ & $2,154\cdot 10^{-3}$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $-4,220\cdot 10^{-1} \sin(x_1) + 9,624\cdot 10^{-1} \sin(1 x3)$ & $2,680\cdot 10^{-4}$ \\
Trygonometryczna (st. 1) & 0,1 & $-4,212\cdot 10^{-1} \sin(x_1) + 9,621\cdot 10^{-1} \sin(1 x3)$ & $1,022\cdot 10^{-2}$ \\
\hline
Wielomiany (st. 3) & 1e-05 & $-4,208\cdot 10^{-1} x_1 + 9,604\cdot 10^{-1} x3\dots$ & $3,168E-25

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [ ]:
x_sim = model.simulate(x0=x0_val, t=t)
mse = ((x_sim - x_val)**2).mean(axis=0)
print(f'Błąd średniokwadratowy x1: {mse[0]}, x2: {mse[1]}, x3: {mse[2]}')

In [ ]:
plt.plot(t, x_val[:, 0])
plt.plot(t, x_val[:, 1])
plt.plot(t, x_val[:, 2], 'y')
plt.legend(["Kąt natarcia", "Kąt nachylenia", "Współczynnik nachylenia"])
plt.xlim(0, max(t))
plt.xlabel("Czas [s]")
plt.ylabel("Rad")

plt.grid()
plt.show()